In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "3"
import torch
from transformers import (
    LlamaConfig, LlamaForCausalLM, LlamaTokenizer
)
from utils.fallacy_utils import *
from utils.format_utils import format_tokens
import pprint
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
argo_fal = [
'Appeal to Emotion',
'Red Herring',
'Hasty Generalization',
'Ad Hominem',
'Appeal to False Authority'
]
print(FALLACY_DEFINITIONS)

{'Ad Hominem': "a fallacy when someone attacks the others' characters or motives instead of addressing the substance of their arguments.", 'Appeal to Opinion of False Authority': "a fallacy when someone attempts to argue or persuade by referring to the opinions or statements of another questionable authority who lacks sufficient credibility in the discussed matter because the authority's expertise may be inadequate/irrelevant or the authority is attributed a statement that has been tweaked.", 'Red Herring': 'a fallacy when someone introduces irrelevant or confusing information in arguments to diverge attention from the main topic being discussed to irrelevant issues.', 'Emotional Language': 'a fallacy when someone attempts to argue or persuade by using strong emotionally charged language with apparent emotional inclination to arouse non-rational sentiments within the intended audience.', 'Hasty Generalization': 'a fallacy when someone makes generalizations based on partial/incomplete o

In [2]:
model_id = './models/llama2_hf/13bf'
tokenizer = LlamaTokenizer.from_pretrained(model_id)
kwargs = {'load_in_8bit': True, 'torch_dtype':torch.float16, 'low_cpu_mem_usage':True}#'device_map':'auto', 
model = LlamaForCausalLM.from_pretrained(model_id, **kwargs)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [54]:
kwargs = {
    "max_new_tokens": 512,
    "do_sample" : True,
    "top_p": 0.9,#0.9, 0.5
    "temperature": 0.75,#0.75, 0.5
    "use_cache": True,
    "top_k" : 50,#50,30
    "repetition_penalty": 1.,
    "length_penalty": 1
}
def MultiturnChat(tokenizer, model, prompt, chat_history):
    #print(chat_history)
    def format_prompt(prompt, chat_history):
        SYSTEM_PROMPT = [
            '''You are a knowledgable expert in detecting fallacies in arguments. 
            Please ensure that your responses are socially unbiased and positive in nature.
            If you don't know the answer to a question, please don't share false information.
            Answer the last question.''',
            '''You are a knowledgable expert in analysing fallacies in arguments. 
            Please ensure that your responses are socially unbiased in nature.
            Your response should not be lengthy.
            Answer the last question.'''
        ]
        dialog = []
        sys_pt = {"role": "system", "content": SYSTEM_PROMPT[1]}
        usr_pt = {"role": "user", "content": prompt}
        if chat_history:
            dialog = [sys_pt] + chat_history + [usr_pt]
        else:
            dialog = [sys_pt] + [usr_pt]
        chat_history = dialog
        #print(chat_history)        
        formatted_prompt = format_tokens(None, [dialog], tokenizer)[0]
        if dialog[0]["role"] == "system":
            dialog = [
            {
                "role": dialog[1]["role"],
                "content": B_SYS
                + dialog[0]["content"]
                + E_SYS
                + dialog[1]["content"],
            }
            ] + dialog[2:]
        assert all([msg["role"] == "user" for msg in dialog[::2]]) and all([msg["role"] == "assistant" for msg in dialog[1::2]]), (
            "model only supports 'system','user' and 'assistant' roles, "
            "starting with user and alternating (u/a/u/a/u...)")
        """
        Please verify that your tokenizer support adding "[INST]", "[/INST]" to your inputs.
        Here, we are adding it manually.
        """
        formatted_dialog = [
            f"{B_INST} {(prompt['content']).strip()} {E_INST} {(answer['content']).strip()} "
            for prompt, answer in zip(dialog[::2], dialog[1::2])
        ]
        
        assert (
            dialog[-1]["role"] == "user"
        ), f"Last message must be from user, got {dialog[-1]['role']}"
        formatted_dialog += [f"{B_INST} {(dialog[-1]['content']).strip()} {E_INST}"]
            
        return " ".join(formatted_dialog), chat_history
    
    formatted_prompt, chat_history = format_prompt(prompt, chat_history)
    model_input = tokenizer(formatted_prompt, return_tensors="pt").to("cuda")
    model.eval()
    with torch.no_grad():
        response = tokenizer.decode(model.generate(**model_input, **kwargs)[0], skip_special_tokens=True)
        res = response.split(E_INST)
        print("\n".join(res[:-1]))
        print()
        print(pprint.pformat(res[-1]))
        chat_history = chat_history[1:] + [{"role":"assistant", "content":res[-1]}]
        return chat_history

In [74]:
FALLACY_DEFINITIONS = {
"Ad Hominem": "a fallacy when someone attacks the others' characters or motives instead of addressing the substance of their arguments.",
"Appeal to Opinions of False Authority": "a fallacy when someone attempts to argue or persuade by referring to the opinions or statements of another questionable authority figure who lacks sufficient credibility in the discussed matter because the authority's expertise may be inadequate/irrelevant or the authority is attributed a statement that has been tweaked.",
"Red Herring": "a fallacy when someone introduces irrelevant or confusing information in arguments to diverge attention from the main topic being discussed to irrelevant issues.",
"Appeal to Emotion": "a fallacy when someone attempts to argue or persuade by using emotive language to arouse non-rational sentiments within the intended audience.",
"Hasty Generalization": "a fallacy when someone makes generalizations based on partial/incomplete observations on a small sample of the whole populations that cannot represent or generalize to other situations legitimately."
}
argo_fal = list(FALLACY_DEFINITIONS.keys())
BASELINE_TEMPLATES =[
'''Review the definitions of these fallacies,
{definitions}
Review the conversation,
{dialog}
Based on all your previous analysis, determine which of the fallacies defined above is most dominantly present in B's argument replied to A?
Output your answer in JSON format {{"fallacy": name_of_the_fallacy, "explanation": in_a_sentence_or_two}}. If none of the fallacies is found, output {{"fallacy": "No Fallacy", "explanation": in_a_sentence_or_two}}. Only output JSON.
''',
'''Based on the following definitions of fallacies,
{definitions}
Given the conversation below,
{dialog}
Determine whether any of the fallacies defined above is present in B's argument replied to A?
Output your answer in JSON format {{"fallacy": name_of_the_fallacy, "explanation": in_a_sentence_or_two}}. If none of the fallacies is found, output {{"fallacy": "No Fallacy", "explanation": in_a_sentence_or_two}}. Only output JSON.
'''
]
#about Hasty Generalization, Appeal to Emotion, Ad Hominem, Red Herring and Appeal to False Authority
def get_USER_PROMPTS(qa):
    #ASK_SINGLE_FALLACY = """According to the definition of {name}: "{defi}", considering B's stance, determine whether this fallacy is present or absent in B's argument."""
    #ASK_SINGLE_FALLACY = """According to the definition of {name}: "{defi}", review the conversation again "{dialog}", is this fallacy present in B's argument?"""
    ASK_SINGLE_FALLACY = """According to the definition of {name}: "{defi}", given the conversation below: \n{dialog}\n, is the fallacy defined above present or absent in B's argument?"""
    def_str = "\n".join([f"{str(i+1)}. {fal} is {FALLACY_DEFINITIONS[fal]}" for i, fal in enumerate(argo_fal)]) 
    USER_PROMPTS = {
    #0:"""Given the conversation below,\n{dialog}\nIs B in favor of or against the topic proposed by A?""".format(dialog=qa),
    #5:"""Based on all your previous analysis, output all the fallacy/fallacies present in B's argument in JSON format {{"fallacy": [name(s)_of_the_fallacy]}}. If none of the fallacies is present, output {{"fallacy": "No Fallacy"}}. Only output JSON."""
    #5:"""Based on all your previous analysis, determine which of the fallacies defined above is dominantly present in B's argument replied to A? Output the fallacy in JSON format {{"fallacy": name_of_the_fallacy}}. If none of the fallacies is present, output {{"fallacy": "No Fallacy"}}. Only output JSON."""
    5: BASELINE_TEMPLATES[1].format(definitions=def_str, dialog=qa)
    }#1:"""Considering B's stance, is it logically sound for B to draw an argument like that?""",
    def_questions = {}
    for i, (name, def_) in enumerate(FALLACY_DEFINITIONS.items()):
        def_questions[i] = ASK_SINGLE_FALLACY.format(name=name, defi=def_, dialog=qa)
    USER_PROMPTS.update(def_questions)
    return USER_PROMPTS

In [75]:
# Test examples
qa0="A: Is democracy the best system available?\nB: good point. why not" # no fallacy??
qa1="A: Is veganism the best for our bodies?\nB: humans are omnivores, we need both meat and vegetables." #ad homien??
qa2="A: Has anyone been on the moon?\nB: The moon is so far away, we should focus on our society." # red hering
qa3 = "A: Is global warming really an issue?\nB: Yes, all the polar-bears are dying, and we are next."
qa4 = "A: Is there a good reason for the American war on terror?\nB: If you fight once you will never stop fighting. Never start!"
qa5 = "A: Is it justified to develop nuclear energy for commercial use?\nB: The green party in Germany has the opinion, that nuclear reactors are bad for us. It is not ok" # false authority
qa6 = '''A: Should smoking at home be illegal?\nB: Home is the place of love and peace, how can you fog it with smoke?'''#appeal to emotion
qa7 = "A: Do we need a global environment task force?\nB: It is people like you who force us normal people into the dictatorship of unnecessary organizations." #ad hominem
qa8 = "A: Is it effective to censor parts of the media?\nB: It is effective to censor parts of the media, because you never know who is watching."
qa9 = "A: Do we need a global environment task force?\nB: The global taskforce for human rights is doing a good job. Thats why we need one for environment,  too."
qa10 = "A: Do we need a global environment task force?\nB: Yes, we need. There're some important enviroment issues that need global efforts to address."
qa11 = "A: Should animals have special rights?\nB: Regarding this argumentation you have the IQ of an insect. Do you think you need special rights?" #ad hominem
qa_emotion = "A: Is nuclear power a reasonable energy source?\nB: Nuclear power was the reason of death of millions of people. It should vanish from this planet."

prompts = get_USER_PROMPTS(qa5)
print(prompts)

{5: 'Based on all your previous analysis, determine which of the fallacies defined above is dominantly present in B\'s argument replied to A? Output the fallacy in JSON format {{"fallacy": name_of_the_fallacy}}. If none of the fallacies is present, output {{"fallacy": "No Fallacy"}}. Only output JSON.', 0: 'According to the definition of Ad Hominem: "a fallacy when someone attacks the others\' characters or motives instead of addressing the substance of their arguments.", given the conversation below: \nA: Is it justified to develop nuclear energy for commercial use?\nB: The green party in Germany has the opinion, that nuclear reactors are bad for us. It is not ok\n, is the fallacy defined above present or absent in B\'s argument?', 1: 'According to the definition of Appeal to Opinions of False Authority: "a fallacy when someone attempts to argue or persuade by referring to the opinions or statements of another questionable authority figure who lacks sufficient credibility in the discu

In [76]:
chat_history=[]

In [82]:
chat_history = MultiturnChat(tokenizer, model, prompts[5], chat_history=chat_history)

[INST] <<SYS>>
You are a knowledgable expert in analysing fallacies in arguments. 
            Please ensure that your responses are socially unbiased in nature.
            Your response should not be lengthy.
            Answer the last question.
<</SYS>>

According to the definition of Ad Hominem: "a fallacy when someone attacks the others' characters or motives instead of addressing the substance of their arguments.", given the conversation below: 
A: Is it justified to develop nuclear energy for commercial use?
B: The green party in Germany has the opinion, that nuclear reactors are bad for us. It is not ok
, is the fallacy defined above present or absent in B's argument? 
 Based on the conversation provided, the fallacy defined above (Ad Hominem) is absent in B's argument. B is not attacking the character or motives of the Green Party in Germany, but rather presenting their opinion as a reason for why nuclear energy may not be justified for commercial use. B's argument is address

In [12]:
chat_history = MultiturnChat(tokenizer, model, prompt_emo, chat_history=chat_history)

[INST] <<SYS>>
You are a knowledgable expert in analysing fallacies in arguments. 
            Please ensure that your responses are socially unbiased in nature.
            If you don't know the answer to a question, please don't share false information.
            Your response should not be lengthy.
            Answer the last question.
<</SYS>>


According to the definition of Emotional Language: "A fallacy when someone tries to to persuade his stance using emotive language to arouse non-rational emotions within the intended audience.,
given a conversation, determine whether this fallacy is present or absent in B's argument and output in JSON format.
Example 1
Conversation:
A: Is nuclear power a reasonable energy source?
B: Nuclear power was the reason of death of millions of people. It should vanish from this planet.
Answer:
{"decision": "Present", "explanation": "B's argument does commit the fallacy of Emotional Language. B attempts to arouse potential negative emotions (e.g., f

In [10]:
prompt_emo = '''
According to the definition of Emotional Language: "A fallacy when someone tries to to persuade his stance using emotive language to arouse non-rational emotions within the intended audience.,
given a conversation, determine whether this fallacy is present or absent in B's argument and output in JSON format.
Example 1
Conversation:
A: Is nuclear power a reasonable energy source?\nB: Nuclear power was the reason of death of millions of people. It should vanish from this planet.
Answer:
{{"decision": "Present", "explanation": "B's argument does commit the fallacy of Emotional Language. B attempts to arouse potential negative emotions (e.g., fear, anger, and aversion) towards nuclear power within people by stating that nuclear power has caused many deaths. B’s argument is an emotive claim that is not supported by evidence, hence it is not a reasoned argument against the use of nuclear power."}}
Example 2
A: Do we need a global environment task force?\nB: The global taskforce for human rights is doing a good job. Thats why we need one for environment, too.
Answer: 
{{"decision": "Absent", "explanation": "B's argument does not commit the fallacy of Emotional Language. B's argument does not employ emotionally charged language to persuade his stance. Therefore, the Emotional Language fallacy is absent."}}
---------------
Conversation:
{dialog}
'''.format(dialog=qa5)

In [ ]:
print(chat_history)

In [2]:
ASK_SINGLE_FALLACY = """According to the definition of {name}: "{defi}", given the conversation below: \n"[]"\n, is the fallacy defined above present or absent in B's argument?"""

In [4]:
tp = ASK_SINGLE_FALLACY.format(name="a", defi="b")
if "\"[]\"" in tp:
    tp = tp.replace("\"[]\"", "\"{dialog}\"")
usr_ct = tp.format(dialog='sdfadff') if '{dialog}' in tp else tp
usr_ct

'According to the definition of a: "b", given the conversation below: \n"sdfadff"\n, is the fallacy defined above present or absent in B\'s argument?'